In [17]:
from collections import Counter

In [1]:
import pandas as pd
from tqdm.notebook import tqdm

In [2]:
tqdm.pandas()

/home/daniil_boiko/anaconda3/lib/python3.8/site-packages/tqdm/std.py:697: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [3]:
from pymystem3 import Mystem
from pyaspeller import YandexSpeller
from wiki_ru_wordnet import WikiWordnet

mystem = Mystem()
wikiwordnet = WikiWordnet()
speller = YandexSpeller()

In [4]:
QUERY_FILE = '/home/daniil_boiko/data/queries.tsv'

In [5]:
data = pd.read_csv(QUERY_FILE, sep='\t', header=None)

In [6]:
data.columns = ['id', 'query']

In [7]:
data['corrected'] = data['query'].progress_map(lambda x: speller.spelled(x))

In [14]:
data['lemmatized'] = data['corrected'].progress_map(lambda x: ''.join(mystem.lemmatize(x)).strip())

In [20]:
def get_synonyms(token):
    return Counter([word.lemma() for synset in wikiwordnet.get_synsets(token) for word in synset.get_words()])

In [25]:
data['synonyms'] = data['lemmatized'].progress_map(
    lambda x: {token:get_synonyms(token) for token in set(x.split(' '))}
)

In [40]:
from nltk.corpus import stopwords
russian_stopwords = stopwords.words("russian")

In [44]:
data['lemmatized_wo_stop_words'] = data['lemmatized'].progress_map(
    lambda x: ' '.join([token for token in x.split(' ') if token not in russian_stopwords])
)

In [46]:
import pickle as pkl

with open('data/queries.corrected.pkl', 'wb') as f:
    pkl.dump(data, f)